In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator
import random
import numpy as np
import pandas as pd
from typing import List, Dict, Union
import yaml
from bisect import bisect_left
from tqdm import tqdm
import calendar
import os
import math
from PIL import Image
import cv2
# from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration, AutoConfig
import torch
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import gc
import warnings
warnings.filterwarnings("ignore")

2023-07-05 01:14:21.601945: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-07-05 01:14:21.603235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/li

In [2]:
chart_type = "histogram"
version = 1

ver1.1 : gtの小数点を丸め処理

In [3]:
save_dir = f"./../input/benetech-synthesis-4th/{chart_type}_images_ver{version}"
os.makedirs(save_dir, exist_ok=True)
processor = AutoProcessor.from_pretrained("google/deplot")

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [5]:
def round_float(value: Union[int, float, str]) -> Union[str, float]:
    if isinstance(value, float):
        value = str(value)

        if "." in value:
            integer, decimal = value.split(".")
            if abs(float(integer)) >= 100:
                return integer
            elif abs(float(integer)) < 100 and abs(float(integer)) >= 10:
                decimal = decimal[:1]
            elif abs(float(integer)) < 10 and abs(float(integer)) >= 1:
                decimal = decimal[:2]
            else:
                decimal = decimal[:4]

            value = integer + "." + decimal
    return value

In [6]:
def get_x_values(num, bins):
    
    y_list = []
    r = np.random.rand()
            
    # 0固定 ~ 10小数
    if r < 0.1:
        a = 0
        b = bins
        
    # 0固定 ~ 10整数
    elif r < 0.3:
        a = 0
        b = 2*bins
        
    # 0固定 ~ 100 整数
    elif r < 0.4:
        a = 0
        b = 5*bins
        
    # 0固定 ~ 10000整数
    elif r < 0.6:
        a = 0
        b = 10*bins
        
    # 0 ~ 10000 整数
    elif r < 0.7:
        a = 0
        b = 20*bins
        
    # 100000~ 10000000 整数
    else:
        a = 0
        b = 100*bins
        
    for _ in range(num):
        y = random.randint(a, b)
        y_list.append(y)
        
    bin_edges = [b//bins*i for i in range(bins)]
    
    return y_list, bin_edges

In [7]:
# ヒストグラムのxラベルは数字
def get_x_label(source):
    
    if source == "None":
        return ""
    
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [8]:
def get_y_label(source):
    
    if source == "None":
        return ""
    
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [9]:
def get_title(source):
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [10]:
def get_font(source):
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [11]:
def comma_format_func(value, tick_number):
    return f'{int(value):,}'

def dollar_format_func(value, tick_number):
    return f'${int(value):,}'

def yen_format_func(value, tick_number):
    return f'¥{int(value):,}'

def euro_format_func(value, tick_number):
    return f'€{int(value):,}'

def percent_format_func(value, tick_number):
    if type(value) == float:
        return f'{value:.2%}'
    else:
        return f'{value}%'

In [12]:
# with open('graph_generation_ver2.yaml') as file:
with open('graph_generation_ver4.yaml') as file:
    data_gen = yaml.safe_load(file)

In [13]:
def generate_graph(data, idx):
    # データ点数
    x_rand = np.random.rand()
    # 頻度が0のケース多め    
    if x_rand < 0.3:
        bins = random.randint(3, 10)
        num_points = random.randint(bins+1, 50)
            
    # ポイント数普通
    elif x_rand < 0.6:
        bins = random.randint(3, 10)
        num_points = random.randint(bins+1, 500)
            
    # ポイント数多め
    else:
        bins = random.randint(3, 10)
        num_points = random.randint(bins+1, 5000)
        
    
    # データ作成
    x_label_source = random.choice(["numericals.txt" ,"measurements.txt"])
    x_label = get_x_label(x_label_source)
    #print(x_label)
    y_label_source = random.choices(data["numerical_label"]["value"], k=1, weights=data["numerical_label"]["weight"])[0]
    y_label = get_y_label(y_label_source)
    #print(y_label)
    
    x_values, bin_edges = get_x_values(num_points, bins) 
    
    # フォントとフォントサイズの設定
    fontsize = random.choices(data["fontsize"]["value"], k=1, weights=data["fontsize"]["weight"])[0]
    plt.rcParams['font.size'] = fontsize
    
    font_source = random.choices(data["font"]["value"], k=1, weights=data["font"]["weight"])[0]
    font = get_font(font_source)
    #print(font)
        
    # グラフのサイズを指定 例：4.56=456 pixel
    size1 = random.randint(400, 800)
    size2 = random.randint(300, size1)
    size1 /= 100
    size2 /= 100
    fig, ax = plt.subplots(figsize=(size1, size2))
    
    color = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    edgecolor = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    linewidth = random.choices([0,1,2,3], k=1, weights=[0.1, 0.4, 0.4, 0.1])[0]
    
    #ax.bar(x_values, y_values, width=barwidth, color=color, edgecolor=edgecolor, linewidth=linewidth)
    y_values, _, _ = ax.hist(x_values, bins=bin_edges, color=color, edgecolor=edgecolor, linewidth=linewidth)
    y_values = y_values.astype(int)
    
    #  y-tickにフォント設定
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
    
    plt.xticks(bin_edges)
    
    # ラベルにフォント設定
    if x_label:
        ax.set_xlabel(x_label, fontname=font, labelpad=20)
    if y_label:
        ax.set_ylabel(y_label, fontname=font, labelpad=20)
    
    # グラフタイトル
    title_source = random.choices(data["title"]["value"], k=1, weights=data["title"]["weight"])[0]
    if title_source != "None":
        title = get_title(title_source)
        ax.set_title(title, fontname=font)
    else:
        title = ""
    
    # 背景色
    if color != "k":
        background_color = random.choices(data["background_color"]["value"], k=1, weights=data["background_color"]["weight"])[0]
        ax.set_facecolor(background_color)
                
    # 画像の保存
    id = str(idx).zfill(8)
    path = os.path.join(save_dir, f"{id}.jpg")
    fig.savefig(path, bbox_inches='tight')
    
    # 画質を落とす
    image = Image.open(path)
    quality = random.randint(10, 100)
    image.save(path, quality=quality)
    
    # if idx % 1 == 0:
        # plt.show()
        #print(font)
        # print(size1, size2)
    
#     img = cv2.imread(path)
#     print(img.shape)
    
    plt.close()
    #plt.show()
    
    
    # -------------------- gt作成 --------------------
    
    y_values = [int(y) for y in y_values]
    x_values = [str(x) for x in x_values]
        
    # gt文字列を作成
    
    # ヒストグラムだけxがyより多くなる例外ケース
    if len(bin_edges) != len(y_values):
        y_values.append("")
    
    gt_string = []
    for x, y in zip(bin_edges, y_values):
        gt_string.append(f"{x} | {y}")
    gt_string = "<0x0A>".join(gt_string)
    gt_string += processor.tokenizer.eos_token
    
    
    df = pd.DataFrame({"id": idx, "ground_truth": gt_string, "fold": -2, "x": [x_values], "y": [y_values], "source": "synthesis", "chart-type": chart_type, "image_path":path[3:]})
    gc.collect()
    return df

In [14]:
seed_everything(42)
from multiprocessing import Pool
import multiprocessing
num_cores = multiprocessing.cpu_count()
def _generate_graph(idx):
    return generate_graph(data_gen,idx)

num = 1000
args = list(range(num))
run = _generate_graph

with Pool(processes=num_cores) as pool:
    imap = pool.imap_unordered(run,args)
    df_list = list(tqdm(imap, total=len(args)))
df = pd.concat(df_list).reset_index(drop=True)
df.to_csv(f"./../input/benetech-synthesis-4th/graph_synthesis_{chart_type}_{num}_ver{version}.csv", index=False)

100%|██████████| 1000/1000 [00:17<00:00, 56.24it/s]
